In [1]:
import json
import numpy as np
import pandas as pd
import feather

## Utility functions

In [2]:
def replace_quotes(string):
    return string.replace("'", '"')


def replace_quotes_region(string):
    return (
        string.replace("{'percentage': '", '{"percentage": "')
        .replace("', 'region': '", '", "region": "')
        .replace("', 'region': \"", '", "region": "')
        .replace("'}", '"}')
    )


def parse_json_obj(series):
    return series.apply(json.loads)


def get_lower(dictionary):
    lower = dictionary.get("lower_bound", np.nan)

    return pd.to_numeric(lower, errors="ignore")


def get_upper(dictionary):
    upper = dictionary.get("upper_bound", np.nan)

    return pd.to_numeric(upper, errors="ignore")

## Load raw data from Ad Library API

In [3]:
df = (
    pd.read_csv(
        "data/raw/facebook-ad-library/fb-archived-ads-GB-2019-12-16-1542.csv",
        usecols=[*range(0, 17)],
        converters={
            "demographic_distribution": replace_quotes,
            "impressions": replace_quotes,
            "region_distribution": replace_quotes_region,
            "spend": replace_quotes,
        },
        parse_dates=["ad_creation_time", "ad_delivery_start_time"],
    )
    .assign(
        ad_delivery_stop_time=lambda df: pd.to_datetime(
            df["ad_delivery_stop_time"], errors="coerce"
        )
    )
    .assign(
        impressions_lower=lambda df: parse_json_obj(df["impressions"])
        .apply(lambda x: get_lower(x))
        .astype("int64", errors="ignore")
    )
    .assign(
        impressions_upper=lambda df: parse_json_obj(df["impressions"]).apply(
            lambda x: get_upper(x)
        )
    )
    .assign(
        spend_lower=lambda df: parse_json_obj(df["spend"])
        .apply(lambda x: get_lower(x))
        .astype("int64", errors="ignore")
    )
    .assign(
        spend_upper=lambda df: parse_json_obj(df["spend"]).apply(lambda x: get_upper(x))
    )
    .assign(
        ad_id=lambda df: df["ad_snapshot_url"].str.extract(r"id=(\d+)", expand=True)
    )
    .sort_values("ad_delivery_start_time", ascending=False)
    .reset_index(drop=True)
)

print(len(df))
print(df.dtypes)
df.head()

219371
ad_creation_time                datetime64[ns, UTC]
ad_creative_body                             object
ad_creative_link_caption                     object
ad_creative_link_description                 object
ad_creative_link_title                       object
ad_delivery_start_time          datetime64[ns, UTC]
ad_delivery_stop_time           datetime64[ns, UTC]
ad_snapshot_url                              object
currency                                     object
demographic_distribution                     object
funding_entity                               object
impressions                                  object
page_id                                       int64
page_name                                    object
publisher_platforms                          object
region_distribution                          object
spend                                        object
impressions_lower                             int64
impressions_upper                           float64
spend

,ad_creation_time,ad_creative_body,ad_creative_link_caption,ad_creative_link_description,ad_creative_link_title,ad_delivery_start_time,ad_delivery_stop_time,ad_snapshot_url,currency,demographic_distribution,...,page_id,page_name,publisher_platforms,region_distribution,spend,impressions_lower,impressions_upper,spend_lower,spend_upper,ad_id
0,2019-12-16 13:20:34+00:00,"You donate, we’ll match. Patagonia is matching...",eu.patagonia.com,"You donate, we’ll match.",Donate to Power for People,2019-12-16 13:20:34+00:00,2019-12-22 20:58:16+00:00,https://www.facebook.com/ads/archive/render_ad...,GBP,"[{""percentage"": ""0.153061"", ""age"": ""18-24"", ""g...",...,254451068317816,Patagonia,['facebook'],"[{""percentage"": ""0.639175"", ""region"": ""England...","{""lower_bound"": ""0"", ""upper_bound"": ""99""}",0,999.0,0,99.0,2523262631119889
1,2019-12-16 12:22:26+00:00,Get The Expert View On Shares That Could Thriv...,Missing: ad_creative_link_caption,Expert investment information,5 Shares To Watch In 2020,2019-12-16 12:22:26+00:00,NaT,https://www.facebook.com/ads/archive/render_ad...,GBP,"[{""percentage"": ""0.29661"", ""age"": ""25-34"", ""ge...",...,139193179458411,Hargreaves Lansdown,"['facebook', 'instagram']","[{""percentage"": ""0.806723"", ""region"": ""England...","{""lower_bound"": ""0"", ""upper_bound"": ""99""}",0,999.0,0,99.0,471287250247735
2,2019-12-16 12:22:26+00:00,Five Shares For 2020 That Our Experts Believe ...,Missing: ad_creative_link_caption,Expert investment information,2020 Share Picks To Watch,2019-12-16 12:22:26+00:00,NaT,https://www.facebook.com/ads/archive/render_ad...,GBP,"[{""percentage"": ""0.003344"", ""age"": ""55-64"", ""g...",...,139193179458411,Hargreaves Lansdown,"['facebook', 'instagram']","[{""percentage"": ""0.842809"", ""region"": ""England...","{""lower_bound"": ""0"", ""upper_bound"": ""99""}",0,999.0,0,99.0,992770711097338
3,2019-12-16 12:22:26+00:00,Get The Expert View On Shares That Could Thriv...,Missing: ad_creative_link_caption,Expert investment information,5 Shares To Watch In 2020,2019-12-16 12:22:26+00:00,NaT,https://www.facebook.com/ads/archive/render_ad...,GBP,"[{""percentage"": ""0.081081"", ""age"": ""55-64"", ""g...",...,139193179458411,Hargreaves Lansdown,"['facebook', 'instagram']","[{""percentage"": ""0.812081"", ""region"": ""England...","{""lower_bound"": ""0"", ""upper_bound"": ""99""}",0,999.0,0,99.0,1002083696794101
4,2019-12-16 12:22:26+00:00,Five Shares For 2020 That Our Experts Believe ...,Missing: ad_creative_link_caption,Expert investment information,2020 Share Picks To Watch,2019-12-16 12:22:26+00:00,NaT,https://www.facebook.com/ads/archive/render_ad...,GBP,"[{""percentage"": ""0.041597"", ""age"": ""45-54"", ""g...",...,139193179458411,Hargreaves Lansdown,"['facebook', 'instagram']","[{""percentage"": ""0.9046"", ""region"": ""England""}...","{""lower_bound"": ""0"", ""upper_bound"": ""99""}",0,999.0,0,99.0,2543707759242575


### Check for duplicate ad IDs

In [4]:
any(df["ad_id"].duplicated())

False

## Unpack `demographic_distribution`

In [5]:
df_demos = (
    pd.concat(
        (
            pd.io.json.json_normalize(json.loads(x))
            for x in df["demographic_distribution"]
            if x and x != '[{"percentage": "1"}]'
        ),
        keys=df["ad_id"],
        sort=False,
    )
    .astype({"percentage": "float64"})
    .reset_index(level=1, drop=True)
    .assign(demographic=lambda df: df.apply(lambda x: f"{x.gender}_{x.age}", axis=1))
    .pivot(columns="demographic", values="percentage")
    .fillna(0)
    .assign(most_viewed_by=lambda df: df.idxmax(axis=1))
    .merge(
        df[
            [
                "ad_id",
                "ad_creation_time",
                "ad_creative_body",
                "ad_delivery_start_time",
                "ad_delivery_stop_time",
                "ad_snapshot_url",
                "funding_entity",
                "page_id",
                "page_name",
                "publisher_platforms",
                "impressions_lower",
                "impressions_upper",
                "spend_lower",
                "spend_upper",
            ]
        ],
        how="left",
        on="ad_id",
    )
)

print(len(df_demos))
print(df_demos.dtypes)
df_demos.head()

219339
ad_id                                  object
female_13-17                          float64
female_18-24                          float64
female_25-34                          float64
female_35-44                          float64
female_45-54                          float64
female_55-64                          float64
female_65+                            float64
male_13-17                            float64
male_18-24                            float64
male_25-34                            float64
male_35-44                            float64
male_45-54                            float64
male_55-64                            float64
male_65+                              float64
unknown_13-17                         float64
unknown_18-24                         float64
unknown_25-34                         float64
unknown_35-44                         float64
unknown_45-54                         float64
unknown_55-64                         float64
unknown_65+                

,ad_id,female_13-17,female_18-24,female_25-34,female_35-44,female_45-54,female_55-64,female_65+,male_13-17,male_18-24,...,ad_delivery_stop_time,ad_snapshot_url,funding_entity,page_id,page_name,publisher_platforms,impressions_lower,impressions_upper,spend_lower,spend_upper
0,1000002613710226,0.00000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2019-12-12 21:30:00+00:00,https://www.facebook.com/ads/archive/render_ad...,The Conservative Party,8807334278,Conservatives,"['facebook', 'instagram']",0,999.0,0,99.0
1,1000013573721676,0.19457,0.018100,0.013575,0.000000,0.027149,0.040724,0.036199,0.285068,0.054299,...,2019-05-02 17:50:43+00:00,https://www.facebook.com/ads/archive/render_ad...,Northenden World Admin,1692130001038191,Northenden's World,"['facebook', 'instagram']",0,999.0,0,99.0
2,1000015213536502,0.00000,0.129930,0.103248,0.061485,0.089327,0.068445,0.064965,0.000000,0.124130,...,2019-03-31 16:00:49+00:00,https://www.facebook.com/ads/archive/render_ad...,Chris Hughes,447416549036263,"Barrow, Sileby and District Branch Labour Party",['facebook'],1000,4999.0,0,99.0
3,1000031753675382,0.00000,0.177273,0.125325,0.016234,0.000000,0.000000,0.000000,0.000000,0.311039,...,NaT,https://www.facebook.com/ads/archive/render_ad...,the Liberal Democrats,5883973269,Liberal Democrats,"['facebook', 'instagram']",1000,1999.0,0,99.0
4,1000033130338557,0.00000,0.120389,0.050831,0.010138,0.004224,0.002112,0.001690,0.000000,0.459307,...,2019-10-15 08:25:58+00:00,https://www.facebook.com/ads/archive/render_ad...,Hayley Ann Mason,608984755788324,The Card Zoo,"['facebook', 'instagram']",5000,9999.0,0,99.0


## Unpack `region_distribution`

In [6]:
uk_regions = ["England", "Northern Ireland", "Scotland", "Wales"]

df_regions = (
    pd.concat(
        (
            pd.io.json.json_normalize(json.loads(x))
            for x in df["region_distribution"]
            if x and x != '[{"percentage": "1"}]'
        ),
        keys=df["ad_id"],
        sort=False,
    )
    .astype({"percentage": "float64"})
    .query("region in @uk_regions")
    .reset_index(level=1, drop=True)
    .pivot(columns="region", values="percentage")
    .fillna(0)
    .assign(most_viewed_in=lambda df: df.idxmax(axis=1))
    .merge(
        df[
            [
                "ad_id",
                "ad_creation_time",
                "ad_creative_body",
                "ad_delivery_start_time",
                "ad_delivery_stop_time",
                "ad_snapshot_url",
                "funding_entity",
                "page_id",
                "page_name",
                "publisher_platforms",
                "impressions_lower",
                "impressions_upper",
                "spend_lower",
                "spend_upper",
            ]
        ],
        how="left",
        on="ad_id",
    )
)

print(len(df_regions))
print(df_regions.dtypes)
df_regions.head()

219344
ad_id                                  object
England                               float64
Northern Ireland                      float64
Scotland                              float64
Wales                                 float64
most_viewed_in                         object
ad_creation_time          datetime64[ns, UTC]
ad_creative_body                       object
ad_delivery_start_time    datetime64[ns, UTC]
ad_delivery_stop_time     datetime64[ns, UTC]
ad_snapshot_url                        object
funding_entity                         object
page_id                                 int64
page_name                              object
publisher_platforms                    object
impressions_lower                       int64
impressions_upper                     float64
spend_lower                             int64
spend_upper                           float64
dtype: object


,ad_id,England,Northern Ireland,Scotland,Wales,most_viewed_in,ad_creation_time,ad_creative_body,ad_delivery_start_time,ad_delivery_stop_time,ad_snapshot_url,funding_entity,page_id,page_name,publisher_platforms,impressions_lower,impressions_upper,spend_lower,spend_upper
0,1000002613710226,1.000000,0.000000,0.000000,0.000000,England,2019-12-08 13:06:31+00:00,A vote for anyone other than the Conservatives...,2019-12-08 13:06:31+00:00,2019-12-12 21:30:00+00:00,https://www.facebook.com/ads/archive/render_ad...,The Conservative Party,8807334278,Conservatives,"['facebook', 'instagram']",0,999.0,0,99.0
1,1000013573721676,1.000000,0.000000,0.000000,0.000000,England,2019-05-01 17:50:43+00:00,This is part of an electioneering leaflet whic...,2019-05-01 17:50:47+00:00,2019-05-02 17:50:43+00:00,https://www.facebook.com/ads/archive/render_ad...,Northenden World Admin,1692130001038191,Northenden's World,"['facebook', 'instagram']",0,999.0,0,99.0
2,1000015213536502,1.000000,0.000000,0.000000,0.000000,England,2019-03-21 16:00:49+00:00,On Thursday 2 May voters in Quorn and Mountsor...,2019-03-21 16:00:54+00:00,2019-03-31 16:00:49+00:00,https://www.facebook.com/ads/archive/render_ad...,Chris Hughes,447416549036263,"Barrow, Sileby and District Branch Labour Party",['facebook'],1000,4999.0,0,99.0
3,1000031753675382,1.000000,0.000000,0.000000,0.000000,England,2019-11-28 12:22:35+00:00,The Liberal Democrats can beat Labour in seats...,2019-11-28 12:22:35+00:00,NaT,https://www.facebook.com/ads/archive/render_ad...,the Liberal Democrats,5883973269,Liberal Democrats,"['facebook', 'instagram']",1000,1999.0,0,99.0
4,1000033130338557,0.734269,0.067319,0.115788,0.080357,England,2019-10-10 08:25:58+00:00,"Sorry Not Sorry! Leave, Remain or Couldn't Car...",2019-10-10 08:25:59+00:00,2019-10-15 08:25:58+00:00,https://www.facebook.com/ads/archive/render_ad...,Hayley Ann Mason,608984755788324,The Card Zoo,"['facebook', 'instagram']",5000,9999.0,0,99.0


## Write to `.feather`

In [7]:
feather.write_dataframe(df, "data/processed/fb-ads-2019-12-16.feather")

In [8]:
feather.write_dataframe(
    df_demos, "data/processed/fb-ads-demographics-2019-12-16.feather"
)

In [9]:
feather.write_dataframe(df_regions, "data/processed/fb-ads-regions-2019-12-16.feather")